In [1]:
from data import get_data, create_pairs
from enlarge_data import get_enlarge_data
from model import *
from train import train
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
img_size = (100, 100)

In [3]:
data, train_indices, test_indices, val_indices = get_data(img_size=img_size, preprocessing='None')
train_pairs, train_labels = create_pairs(data, train_indices)
test_pairs, test_labels = create_pairs(data, test_indices)
val_pairs, val_labels = create_pairs(data, val_indices)

print(f"{np.array(data).shape = }")
print(f"{train_pairs.shape = }, {train_labels.shape = }")
print(f"{test_pairs.shape = }, {test_labels.shape = }")
print(f"{val_pairs.shape = }, {val_labels.shape = }")

100%|██████████| 40/40 [00:34<00:00,  1.17it/s]

np.array(data).shape = (1600, 100, 100, 3)
train_pairs.shape = (2160, 2, 100, 100, 3), train_labels.shape = (2160,)
test_pairs.shape = (560, 2, 100, 100, 3), test_labels.shape = (560,)
val_pairs.shape = (240, 2, 100, 100, 3), val_labels.shape = (240,)


In [4]:
model = train(
    train_pairs.shape[-3:],
    simple_model,
    train_pairs,
    train_labels,
    validation_data = ([val_pairs[:, 0], val_pairs[:, 1]], val_labels),
    epochs = 25
)

Epoch 1/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 15s 812ms/step - binary_accuracy: 0.6251 - loss: 0.6892 - val_binary_accuracy: 0.6814 - val_loss: 0.6579
Epoch 2/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 846ms/step - binary_accuracy: 0.7220 - loss: 0.5528 - val_binary_accuracy: 0.6802 - val_loss: 0.6782
Epoch 3/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 15s 867ms/step - binary_accuracy: 0.7716 - loss: 0.4873 - val_binary_accuracy: 0.7093 - val_loss: 0.5861
Epoch 4/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - binary_accuracy: 0.8154 - loss: 0.3879 - val_binary_accuracy: 0.7723 - val_loss: 0.5176
Epoch 5/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 908ms/step - binary_accuracy: 0.8278 - loss: 0.3752 - val_binary_accuracy: 0.7757 - val_loss: 0.4576
Epoch 6/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 948ms/step - binary_accuracy: 0.8943 - loss: 0.2868 - val_binary_accuracy: 0.8097 - val_loss: 0.4227
Epoch 7/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 15s 887ms/step - binary_accuracy: 0.9133 - loss: 0.2519 - val_binary_accuracy: 0.7885 - val_loss: 0.4206
Ep

In [15]:
lambda_regularizer = 0.0

test_model = tf.keras.Sequential([
    Resizing(100, 100, interpolation='bilinear'),
    
    
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(lambda_regularizer)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(lambda_regularizer)),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(lambda_regularizer)),
    MaxPooling2D((2, 2)),
    
    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(lambda_regularizer)),
    MaxPooling2D((2, 2)),
    
    Conv2D(512, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(lambda_regularizer)),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    # Dropout(0.2),
    
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
])

In [27]:
weights = model.get_weights()
new_model = dual_model(train_pairs.shape[-3:], test_model)
new_model.set_weights(weights)

for i in range(len(weights) - 1):
    if np.all(weights[i] == test_model.get_weights()[i]):
        print(f"Layer {i} is the same")

Layer 0 is the same
Layer 1 is the same
Layer 2 is the same
Layer 3 is the same
Layer 4 is the same
Layer 5 is the same
Layer 6 is the same
Layer 7 is the same
Layer 8 is the same
Layer 9 is the same
Layer 10 is the same
Layer 11 is the same
Layer 12 is the same
Layer 13 is the same


In [8]:
print(model.summary())

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 32)        │  1,638,368 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction_layer_2  │ (None)            │          1 │ sequential[0][0], │
│ (PredictionLayer)   │                   │            │ sequential[1][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,915,109 (18.75 MB)

 Trainable params: 1,638,369 (6.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,276,740 (12.50 MB)

None


In [5]:
def get_accuracy(model, pairs, labels):
    predictions = model.predict([pairs[:, 0], pairs[:, 1]])
    return accuracy_score(labels, predictions > 0.5)


def print_accuracies(model):
    print(f"Train accuracy: {get_accuracy(model, train_pairs, train_labels)}")
    print(f"Test accuracy: {get_accuracy(model, test_pairs, test_labels)}")
    print(f"Val accuracy: {get_accuracy(model, val_pairs, val_labels)}")

In [6]:
print_accuracies(model)

68/68 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step
Train accuracy: 1.0
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step
Test accuracy: 0.8803571428571428
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step
Val accuracy: 0.8625


In [29]:
model.save('models/model.keras')

In [8]:
small_cnn = train(
    train_pairs.shape[-3:],
    simple_model,
    train_pairs,
    train_labels,
    validation_data = ([val_pairs[:, 0], val_pairs[:, 1]], val_labels),
    epochs = 25
)

Epoch 1/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - binary_accuracy: 0.6990 - loss: 1.0303 - val_binary_accuracy: 0.6278 - val_loss: 0.9893
Epoch 2/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - binary_accuracy: 0.7401 - loss: 0.8261 - val_binary_accuracy: 0.6680 - val_loss: 0.8962
Epoch 3/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - binary_accuracy: 0.7791 - loss: 0.7433 - val_binary_accuracy: 0.7048 - val_loss: 0.9092
Epoch 4/25
 5/17 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - binary_accuracy: 0.8011 - loss: 0.6901

In [ ]:
print(small_cnn.summary())

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 16)        │    229,616 │ input_layer_3[0]… │
│ (Sequential)        │                   │            │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction_layer_1  │ (None)            │          0 │ sequential_2[0][… │
│ (PredictionLayer)   │                   │            │ sequential_2[1][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 688,850 (2.63 MB)

 Trainable params: 229,616 (896.94 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 459,234 (1.75 MB)

None


In [ ]:
print_accuracies(small_cnn)

68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
Train accuracy: 0.9990740740740741
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
Test accuracy: 0.825
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Val accuracy: 0.8


In [ ]:
# alexnet = train(img_size + (3,), alexnet_model, train_pairs, train_labels, epochs=20)

In [ ]:
# print(alexnet.summary())

In [ ]:
# print_accuracies(alexnet)